# An introductional notebook to HEP analysis in C++

<p>In this notebook you can find an easy set of commands that show some basic computing techniques commonly used in High Energy Physics (HEP) analyzes.</p>

<p>It also shows how to create an histogram, fill it and draw it. Moreover it is an introduction to [ROOT](https://root.cern.ch/) too. The final output is a plot with the number of leptons.</p>

Based on ATLAS opendata notebooks (http://opendata.atlas.cern/release/2020/documentation/notebooks/intro.html)


The library used is [ROOT](https://root.cern.ch/), a scientific data analysis software framework that provides a large set of functionalities needed to deal with big data processing, statistical analysis, visualisation and storage.

<p>At first we have to include several helpers that will support our analysis:</p>

In [ ]:
#include <iostream>
#include <string>
#include <stdio.h>

Next we have to open the data that we want to analyze. As described above the data is stored in a _*.root_ file. This is a root file containing tracks and calorimeter clusters

In [ ]:
TFile *file = TFile::Open("Data_8TeV.root"); 

The next step is to define a tree named _tree_ to get the data out of the _*.root_ file. The tree in this root file is called "mini". We will then print the contents of the tree to find the names of the variables. 

In [ ]:
TTree *tree = (TTree*) file->Get("mini");
tree->Print()

Activate variables 

In [ ]:
Bool_t e_trig;
Bool_t mu_trig;
Bool_t good_vtx;
UInt_t lep_n;
UInt_t jet_n;
Float_t MET;
Float_t MET_phi;

Float_t lep_pt[10];  
Float_t lep_eta[10];  
Float_t lep_phi[10];  
Float_t lep_E[10];  
Int_t lep_type[10];  
Float_t lep_ptcone30[10];
Float_t lep_etcone20[10];

Float_t jet_pt[10]; 
Float_t jet_eta[10];
Float_t jet_jvf[10];
Float_t jet_mv1[10];

float evtw = -1;

tree->SetBranchAddress("trigE", &e_trig);
tree->SetBranchAddress("trigM", &mu_trig);
tree->SetBranchAddress("hasGoodVertex", &good_vtx);
tree->SetBranchAddress("lep_n", &lep_n);
tree->SetBranchAddress("jet_n", &jet_n);
tree->SetBranchAddress("met_et", &MET);
tree->SetBranchAddress("met_phi", &MET_phi);

tree->SetBranchAddress("lep_pt", &lep_pt);
tree->SetBranchAddress("lep_eta", &lep_eta);
tree->SetBranchAddress("lep_phi", &lep_phi);
tree->SetBranchAddress("lep_E", &lep_E);
tree->SetBranchAddress("lep_type", &lep_type);
tree->SetBranchAddress("lep_ptcone30", &lep_ptcone30);
tree->SetBranchAddress("lep_etcone20", &lep_etcone20);

tree->SetBranchAddress("jet_pt", &jet_pt);
tree->SetBranchAddress("jet_eta", &jet_eta);
tree->SetBranchAddress("jet_jvf", &jet_jvf);
tree->SetBranchAddress("jet_MV1", &jet_mv1);



Create Canvas

In [ ]:
TCanvas *canvas = new TCanvas("Canvas","",800,600);

Create histograms: Leading jet pT and all jets pT 

In [ ]:
TH1F *cutflow = new TH1F("Cutflow","Cutflow; Cut; Events",10,0,10);

TH1F *hist_njets = new TH1F("Number of jets","n-jets; Jet multiplicity; Events",10,0,10);
// Leptons
TH1F *hist_lep_pt = new TH1F("Lepton pT","Lepton pT; pT (GeV); Events",20,1000,100000);
TH1F *hist_lep_ptcone30 = new TH1F("Lepton ptcone30","Lepton ptcone30; ptcone30; Events",10,0,1);
TH1F *hist_lep_etcone20 = new TH1F("Lepton etcone20","Lepton etcone20; etcone20; Events",10,-1,1);
TH1F *hist_lep_eta = new TH1F("Lepton eta","Lepton eta; eta (GeV); Events",10,-4,4);
// Jets
TH1F *hist_jets_pt = new TH1F("Jets pT","Jets pT; pT (GeV); Events",10,0,100000);
TH1F *hist_jets_eta = new TH1F("Jets eta","Jets eta; eta (rad); Events",10,-4,4);
TH1F *hist_jets_JVF = new TH1F("Jets JVF","Jets JVF; JVF; Events",25,0,1);
TH1F *hist_jets_MV1 = new TH1F("Jets MV1 output","Jets MV1 output; MV1 output; Events",10,0,1);
// B-Jets
TH1F *hist_n_bjets = new TH1F("Number b-Jets ","Number of b-Jets MV1 output; Number of b-Jets; Events",20,0,10);
// MET
TH1F *hist_MET = new TH1F("MET ","MET; MET; Events",200,0,200000);
// mT(W)
TH1F *hist_MTW = new TH1F("mT(w) ","mT(W); mT(w); Events",200,0,200000);

# Graficos

Loop and fill histograms

In [ ]:
int nentries, nbytes, i;
nentries = (Int_t)tree->GetEntries();

int cut1 = 0;
int cut2 = 0;
int cut3 = 0;
int cut4 = 0;
int cut5 = 0;
int cut6 = 0;
int cut7 = 0;
int cut8 = 0;


for (i = 0; i < nentries; i++)
{
    nbytes = tree->GetEntry(i);   

    //First cut: Good vertex
    if(!good_vtx) continue;
    cut1++;
    cutflow->Fill(1);

    //Second cut: Trigger
    if(!e_trig && !mu_trig) continue;
    cut2++;
    cutflow->Fill(2);
        
    // Preselection of good leptons                                                                                
    int n_mu=0;
    int n_el=0;
    int n_lep=0;
    int good_lep=0;
    // Histogramas Leptones
  
    //Loop over leptons
    for(unsigned int i=0; i<lep_n; i++){
        hist_lep_pt->Fill(lep_pt[i]);
        hist_lep_ptcone30->Fill(lep_ptcone30[i]/lep_pt[i]);
        hist_lep_etcone20->Fill(lep_etcone20[i]/lep_pt[i]);
        hist_lep_eta->Fill(lep_eta[i]);
        if( lep_pt[i] < 25000.) continue; 
        if( lep_ptcone30[i]/lep_pt[i] > 0.15 ) continue; 
        if( lep_etcone20[i]/lep_pt[i] > 0.15 ) continue;  
        if( lep_type [i]==13 && TMath::Abs(lep_eta[i]) < 2.5 ){
            n_mu++;
            good_lep=i;}
        /*
        To complete: Add electrons and extract the index for the good lepton
        */
        if( lep_type [i]==11 && TMath::Abs(lep_eta[i]) < 2.47) { 
            if(TMath::Abs(lep_eta[i]) < 1.37 || TMath::Abs(lep_eta[i]) > 1.52 ){
                n_el++;
                good_lep=i;}}
        
    }
    n_lep = n_mu + n_el;
    
     //Select events with only 1 good lepton and fill the cutflow histogram 
    //Example:
    //Third cut (one good lepton):
    if(n_lep!=1) continue;
    cutflow->Fill(3); 
    cut3++;
  
    
    int n_jets=0;
    int n_bjets=0;
    
    //Number of jets distribution
    hist_njets->Fill(jet_n);//,evtw);

    //Fourth cut: At least 4 jets
    if(jet_n<4) continue; 
    cutflow->Fill(4); 
    cut4++;

    for(unsigned int j=0; j<jet_n; j++){
        // Histograms
        hist_jets_pt->Fill(jet_pt[j]);
        hist_jets_eta->Fill(jet_eta[j]);
        hist_jets_JVF->Fill(jet_jvf[j]);
        hist_jets_MV1->Fill(jet_mv1[j]);
        // To complete: apply jet cuts to find the good jets
        if(jet_pt[j] < 25000.) continue;
        //Eta cut
        if(TMath::Abs(jet_eta[j]) > 2.5) continue;
        // JVF cleaning         
        if(jet_pt[j] < 50000 && TMath::Abs(jet_eta[j])<2.4 && jet_jvf[j]<0.5)continue;
        n_jets++;
        // cut on 0.7892 MV1 and count the number of b-jets
        if(TMath::Abs(jet_mv1[j]) < 0.7892) continue;
        n_bjets++;
        
        }
    
    //Fifth cut: At least 4 good jets
    if(n_jets<4) continue; 
    cutflow->Fill(5); 
    cut5++;
    
    hist_n_bjets->Fill(n_bjets);
    //Sixth cut: at least one b-jet
    if(n_bjets<2) continue;
    cutflow->Fill(6);
    cut6++;
    
    hist_MET->Fill(MET);
    //Seventh cut: MET > 30 GeV
    if(MET<30000.) continue;
    cutflow->Fill(7);
    cut7++;
    
    // TLorentzVector definitions                                                               
    TLorentzVector Lepton  = TLorentzVector();
    TLorentzVector  MeT  = TLorentzVector();

    //To complete: Lorentz vectors for the lepton and MET. Use SetPtEtaPhiE().
    Lepton.SetPtEtaPhiE(lep_pt[good_lep],lep_eta[good_lep],lep_phi[good_lep],lep_E[good_lep]);
    MeT.SetPtEtaPhiE(MET,0,MET_phi,MET);
    //Calculation of the mTW using TLorentz vectors    
    float mTW = sqrt(2*Lepton.Pt()*MeT.Et()*(1-cos(Lepton.DeltaPhi(MeT))));

    hist_MTW->Fill(mTW);
    //Eight cut: mTW > 30 GeV
    if(mTW>30000){
        cutflow->Fill(8);
        cut8++;
    }
}

std::cout << "Done!" << std::endl;
std::cout << "All events:" << nentries << std::endl;
std::cout << "Cut1:" << cut1 << std::endl;
std::cout << "Cut2:" << cut2 << std::endl;
std::cout << "Cut3:" << cut3 << std::endl;
std::cout << "Cut4:" << cut4 << std::endl;
std::cout << "Cut5:" << cut5 << std::endl;
std::cout << "Cut6:" << cut6 << std::endl;
std::cout << "Cut7:" << cut7 << std::endl;
std::cout << "Cut8:" << cut8 << std::endl;



<p>Draw</p>

In [ ]:
cutflow->Draw("");
canvas->SetLogy();
canvas->Draw();

In [ ]:
hist_njets->Draw();
canvas->SetLogy();
canvas->Draw();

# Lepton

In [ ]:
hist_lep_pt->Draw();
canvas->SetLogy();
canvas->Draw();

In [ ]:
hist_lep_ptcone30->Draw();
canvas->SetLogy();
canvas->Draw();

In [ ]:
hist_lep_etcone20->Draw();
canvas->SetLogy();
canvas->Draw();

In [ ]:
hist_lep_eta->Draw();
canvas->Draw();

In [ ]:
hist_jets_pt->Draw();
canvas->SetLogy();
canvas->Draw();

In [ ]:
hist_jets_eta->Draw();
canvas->Draw();

In [ ]:
hist_jets_JVF->Draw();
canvas->Draw();

In [ ]:
hist_jets_MV1->Draw();
canvas->Draw();

In [ ]:
hist_n_bjets->Draw();
canvas->Draw();

In [ ]:
hist_MET->Draw();
canvas->Draw();

In [ ]:
hist_MTW->Draw();
canvas->Draw();